<h1> Load Data </h1>

In [ ]:
# -- Import --
import sys
sys.path.append("/home/puguix/Desktop/cryptoBot/data_polo/utility/")

import pandas as pd
from get_data import get_historical_from_db
import ta
import numpy as np
from math import *

# - - Load data from database --
df = get_historical_from_db("Binance","ETH/USDT","5m")

print("Data loaded 100%")


<h1>Define indicators </h1>

In [ ]:
# -- Drop all columns we do not need --
df.drop(df.columns.difference(['open','high','low','close','volume']), 1, inplace=True)


# - - UT Bot Alerts - -

def UTBotAlerts(df,sensitivity, period):
    close = df['close']
    xATR = ta.volatility.average_true_range(df['high'], df['low'], close, period)
    nLoss = sensitivity * xATR
    n = close.size
    trailingStop, pos = [0] * n, [0] * n
    above, below = [False] * n, [False] * n
    buy, sell = [False] * n, [False] * n
    
    ema = ta.trend.ema_indicator(close, window=1)
        
    for curr in range(1, n):
        prev = curr - 1
        if (close.iloc[curr] > trailingStop.iloc[prev] and close.iloc[prev] > trailingStop.iloc[prev]):
            trailingStop.iloc[curr] = max(trailingStop.iloc[prev], close.iloc[curr] - nLoss,)
        elif (close.iloc[curr] < trailingStop.iloc[prev] and close.iloc[prev] < trailingStop.iloc[prev]):
            trailingStop.iloc[curr] = min(trailingStop.iloc[prev], close.iloc[curr] + nLoss,)
        elif (close.iloc[curr] > trailingStop.iloc[prev]):
            trailingStop.iloc[curr] = close.iloc[curr] - nLoss
        else:
            trailingStop.iloc[curr] = close.iloc[curr] + nLoss
            
        if (close.iloc[prev] < trailingStop.iloc[prev] and close.iloc[curr] > trailingStop.iloc[prev]):
            pos.iloc[curr] = 1
        elif (close.iloc[prev] > trailingStop.iloc[prev] and close.iloc[curr] < trailingStop.iloc[prev]):
            pos.iloc[curr] = -1
        else:
            pos.iloc[curr] = pos.iloc[prev]
            
        above.iloc[curr] = ema[curr] > trailingStop.iloc[curr] and ema.iloc[prev] < trailingStop.iloc[prev]
        below.iloc[curr] = ema[curr] < trailingStop.iloc[curr] and ema.iloc[prev] > trailingStop.iloc[prev]
        
        buy.iloc[curr] = close[curr] > trailingStop.iloc[curr] and above.iloc[curr]
        sell.iloc[curr] = close[curr] < trailingStop.iloc[curr] and below.iloc[curr]
        
    res = pd.DataFrame({'buy': buy, 'sell': sell})

    df = pd.concat([df, res], ignore_index = True)
    
# - - fonctions utilitaires - -

def lowest (values, current, length):
    min = values.iloc[current]
    for k in range(1, length):
        min = values.iloc[current - k] if values.iloc[current - k] < min else min
    return min

def highest (values, current, length):
    max = values.iloc[current]
    for k in range(1, length):
        max = values.iloc[current - k] if values.iloc[current - k] > max else max
    return max
    
    
# - - STC - -

def STC(df, EEEEEE, BBBB, BBBBB):
    # EEEEEE = length, BBBB = fast length, BBBBB = slow length
    close = df['close']
    n = close.size
    
    CCCCC = [0] * n
    DDD = [0] * n
    DDDDDD = [0] * n
    EEEEE = [0] * n
    
    mColor = ["red"] * n
    
    def AAAA(BBB, BBBB, BBBBB):
        fastMA = ta.trend.ema_indicator(BBB, BBBB)
        slowMA = ta.trend.ema_indicator(BBB, BBBBB)
        return fastMA - slowMA
    
    def AAAAA(EEEEEE, BBBB, BBBBB, curr):
        prev = curr - 1
        AAA = 0.5
        BBBBBB = AAAA(close.iloc[curr], BBBB, BBBBB)
        CCC = lowest(close, curr, 10)
        CCCC = highest(close, curr, 10) - CCC
        CCCCC.iloc[curr] = (BBBBBB - CCC) / CCCC * 100 if CCCC > 0 else CCCCC.iloc[prev]
        DDD.iloc[curr] = CCCCC.iloc[curr] if DDD.iloc[prev] == 0 else DDD.iloc[prev] + AAA * (CCCCC.iloc[prev] - DDD.iloc[prev])
        DDDD = lowest(DDD, EEEEEE)
        DDDDD = highest(DDD, EEEEEE) - DDDD
        DDDDDD.iloc[curr] = (DDD - DDDD) / DDDDD * 100 if DDDDD > 0 else DDDDDD.iloc[prev]
        EEEEE.iloc[curr] = DDDDDD.iloc[curr] if EEEEE.iloc[prev] == 0 else EEEEE.iloc[prev] * AAA * (DDDDDD.iloc[curr] - EEEEE.iloc[prev])
       
    for curr in range(1, n):
        #mAAAAA = EEEEE est fait en place  
        AAAAA(EEEEEE, BBBB, BBBBB, curr)
        mColor.iloc[curr] = "green" if EEEEEE.iloc[curr] > EEEEEE.iloc[curr - 1] else "red"
    
    res = pd.DataFrame({'STC': EEEEE, 'color': mColor})
    
    df = pd.concat([df, res], ignore_index = True)
    
    
# - - Hull Suite - -

def HullSuite(df, length):
    close = df['close']
    
    def HMA(src, length):
        return ta.trend.wma_indicator(2 * ta.trend.wma_indicator(src, window=length / 2, fillna=False) - ta.trend.wma_indicator(src, window=length, fillna=False), window=round(length ** (1/2)), fillna=False)
    
    HULL = HMA(close, length)
    
    hColor = ["red"] * close.size
    for curr in range(1, close.size):
        hColor.iloc[curr] = "green" if HULL.iloc[curr] > HULL.iloc[curr -2] else "red"
    
    df = pd.concat([df, pd.DataFrame({'hColor': hColor, 'hull': HULL})], ignore_index = True)

<h1> Optimizer </h1>

In [ ]:
dfTest = None
dt = None
dt = pd.DataFrame(columns = ['sensitivity','period', 'length', 'fastLength', 'slowLength', 'hLength', 'result'])

dfTest = df.copy()

# -- If you want to run your BackTest on a specific period, uncomment the line below --
#dfTest = df['2021-01-01':]


# -- Condition to LONG market --
def openLongCondition(row):
    return row['buy'] and row['STC'] < 25 and row['color'] == "green" and row['hColor'] == "green"

def closeLongCondition(row, prevRow):
    return prevRow['hull'] < prevRow['close'] and row['hull'] > prevRow['close']
      
# -- Condition to SHORT market --  
def openShortCondition(row):
    return row['sell'] and row['STC'] > 75 and row['color'] == "red" and row['hColor'] == "red"

def closeShortCondition(row, prevRow):
    return prevRow['hull'] > prevRow['close'] and row['hull'] < prevRow['close']

# - - Loops to try parameters - -

for sensitivity in range(1,3):
    for period in range(5, 11):
        for length in range(50, 100, 4):
            for fastLength in range(20, 35, 2):
                for slowLength in range(45, 60,3):
                    for hLength in range(30, 81, 5):
                        
                        wallet = 1000
                        coin = 0
                        orderInProgress = ''
                        nbTrades = 0
                        
                        UTBotAlerts(dfTest, sensitivity, period)
                        STC(dfTest, length, fastLength, slowLength)
                        HullSuite(dfTest, hLength)
                        
                        for index, row in dfTest.iterrows():
                            
                            if index == 0:
                                previousRow = row
                                continue
                            
                            # -- If there is an order in progress --
                            if orderInProgress != '':
                                # -- Check if there is a LONG order in progress --
                                if orderInProgress == 'LONG':
                                    # -- Check Liquidation --
                                    if row['low'] < longLiquidationPrice:
                                        print('/!\ YOUR LONG HAVE BEEN LIQUIDATED the',index)
                                        break
                                    
                                    # -- Check If you have to close the LONG --
                                    if closeLongCondition(row, previousRow) == True:
                                        orderInProgress = ''
                                        closePrice = row['close']
                                        closePriceWithFee = row['close'] - 0.0007 * row['close']
                                        pr_change = (closePriceWithFee - longIniPrice) / longIniPrice
                                        wallet = wallet + wallet*pr_change

                                # -- Check if there is a SHORT order in progress --
                                elif orderInProgress == 'SHORT':
                                    # -- Check Liquidation --
                                    if row['high'] > shortLiquidationPrice:
                                        print('/!\ YOUR SHORT HAVE BEEN LIQUIDATED the',index)
                                        break

                                    # -- Check If you have to close the SHORT --
                                    if closeShortCondition(row, previousRow) == True:
                                        orderInProgress = ''
                                        closePrice = row['close']
                                        closePriceWithFee = row['close'] + 0.0007 * row['close']
                                        pr_change = -(closePriceWithFee - shortIniPrice) / shortIniPrice
                                        wallet = wallet + wallet*pr_change

                            # -- If there is NO order in progress --
                            if orderInProgress == '':
                                # -- Check If you have to open a LONG --
                                if openLongCondition(row, previousRow) == True:
                                    orderInProgress = 'LONG'
                                    closePrice = row['close']
                                    longIniPrice = row['close'] + 0.0007 * row['close']
                                    tokenAmount = (wallet) / row['close']
                                    longLiquidationPrice = longIniPrice - (wallet/tokenAmount)
                                    nbTrades += 1
                                
                                # -- Check If you have to open a SHORT --
                                if openShortCondition(row, previousRow) == True:
                                    orderInProgress = 'SHORT'
                                    closePrice = row['close']
                                    shortIniPrice = row['close'] - 0.0007 * row['close']
                                    tokenAmount = (wallet) / row['close']
                                    shortLiquidationPrice = shortIniPrice + (wallet/tokenAmount)
                                    nbTrades += 1
                            
                            previousRow = row
                                                
                        myrow = {'sensitivity': sensitivity, 'period': period, 'length': length,'fastLength': fastLength, 'slowLength': slowLength, 'hLength': hLength, 'result': coin*dfTest.iloc[len(dfTest)-1]['close'] + wallet, 'Trades': nbTrades}
                        dt = pd.concat([dt,pd.DataFrame(myrow, index=[0])], ignore_index=True)
                        
print(dt.sort_values(by=['result']))
                            

<h1> Run the backtest </h1>

In [ ]:
dfTest = None
dfTest = df.copy()


UTBotAlerts(dfTest, sensitivity, period)
STC(dfTest, length, fastLength, slowLength)
HullSuite(dfTest, hLength)
                        
# -- If you want to run your BackTest on a specific period, uncomment the line below --
#dfTest = df['2021-01-18':]

# -- Definition of dt, that will be the dataset to do your trades analyses --
dt = None
dt = pd.DataFrame(columns=['date', 'position', 'reason',
                           'price', 'frais', 'wallet', 'drawBack'])

# -- You can change variables below --
leverage = 1
wallet = 1000
makerFee = 0.0002
takerFee = 0.0005

# -- Do not touch these values --
initalWallet = wallet
lastAth = wallet
previousRow = dfTest.iloc[0]
stopLoss = 0
takeProfit = 500000
orderInProgress = ''
longIniPrice = 0
shortIniPrice = 0
longLiquidationPrice = 500000
shortLiquidationPrice = 0

# - - Write open and close conditions here - - 

# -- Condition to LONG market --
def openLongCondition(row):
    return row['buy'] and row['STC'] < 25 and row['color'] == "green" and row['hColor'] == "green"

def closeLongCondition(row, prevRow):
    return prevRow['hull'] < prevRow['close'] and row['hull'] > prevRow['close']
      
# -- Condition to SHORT market --  
def openShortCondition(row):
    return row['sell'] and row['STC'] > 75 and row['color'] == "red" and row['hColor'] == "red"

def closeShortCondition(row, prevRow):
    return prevRow['hull'] > prevRow['close'] and row['hull'] < prevRow['close']

# -- Iteration on all your price dataset (df) --
for index, row in dfTest.iterrows():

    # -- If there is an order in progress --
    if orderInProgress != '':
        # -- Check if there is a LONG order in progress --
        if orderInProgress == 'LONG':
            # -- Check Liquidation --
            if row['low'] < longLiquidationPrice:
                print('/!\ YOUR LONG HAVE BEEN LIQUIDATED the',index)
                break
            
            # -- Check If you have to close the LONG --
            if closeLongCondition(row, previousRow) == True:
                orderInProgress = ''
                closePrice = row['close']
                closePriceWithFee = row['close'] - takerFee * row['close']
                pr_change = (closePriceWithFee - longIniPrice) / longIniPrice
                wallet = wallet + wallet*pr_change*leverage

                # -- You can uncomment the line below if you want to see logs --
                # print('Close LONG at',closePrice,"the", index, '| wallet :', wallet,
                #       '| result :', pr_change*100*leverage)


                # -- Check if your wallet hit a new ATH to know the drawBack --
                if wallet > lastAth:
                    lastAth = wallet
                
                # -- Add the trade to DT to analyse it later --
                myrow = {'date': index, 'position': "LONG", 'reason': 'Close Long Market', 'price': closePrice,
                        'frais': takerFee * wallet * leverage, 'wallet': wallet, 'drawBack': (wallet-lastAth)/lastAth}
                #dt = dt.append(myrow, ignore_index=True)
            dt = pd.concat([dt,pd.DataFrame(myrow, index=[0])], ignore_index=True)

        # -- Check if there is a SHORT order in progress --
        elif orderInProgress == 'SHORT':
            # -- Check Liquidation --
            if row['high'] > shortLiquidationPrice:
                print('/!\ YOUR SHORT HAVE BEEN LIQUIDATED the',index)
                break

            # -- Check If you have to close the SHORT --
            if closeShortCondition(row, previousRow) == True:
                orderInProgress = ''
                closePrice = row['close']
                closePriceWithFee = row['close'] + takerFee * row['close']
                pr_change = -(closePriceWithFee - shortIniPrice) / shortIniPrice
                wallet = wallet + wallet*pr_change*leverage

                # -- You can uncomment the line below if you want to see logs --
                # print('Close SHORT at',closePrice,"the", index, '| wallet :', wallet,
                #       '| result :', pr_change*100*leverage)

                # -- Check if your wallet hit a new ATH to know the drawBack --
                if wallet > lastAth:
                    lastAth = wallet

                # -- Add the trade to DT to analyse it later --
                myrow = {'date': index, 'position': "SHORT", 'reason': 'Close Short Market', 'price': closePrice,
                        'frais': takerFee * wallet * leverage, 'wallet': wallet, 'drawBack': (wallet-lastAth)/lastAth}
                #dt = dt.append(myrow, ignore_index=True)
            dt = pd.concat([dt,pd.DataFrame(myrow, index=[0])], ignore_index=True)

    # -- If there is NO order in progress --
    if orderInProgress == '':
        # -- Check If you have to open a LONG --
        if openLongCondition(row, previousRow) == True:
            orderInProgress = 'LONG'
            closePrice = row['close']
            longIniPrice = row['close'] + takerFee * row['close']
            tokenAmount = (wallet * leverage) / row['close']
            longLiquidationPrice = longIniPrice - (wallet/tokenAmount)

            # -- You can uncomment the line below if you want to see logs --
            # print('Open LONG at', closePrice, '$ the', index, '| Liquidation price :', longLiquidationPrice)

            # -- Add the trade to DT to analyse it later --
            myrow = {'date': index, 'position': "Open Long", 'reason': 'Open Long Market', 'price': closePrice,
                     'frais': takerFee * wallet * leverage, 'wallet': wallet, 'drawBack': (wallet-lastAth)/lastAth}
            #dt = dt.append(myrow, ignore_index=True)
            dt = pd.concat([dt,pd.DataFrame(myrow, index=[0])], ignore_index=True)
        
        # -- Check If you have to open a SHORT --
        if openShortCondition(row, previousRow) == True:
            orderInProgress = 'SHORT'
            closePrice = row['close']
            shortIniPrice = row['close'] - takerFee * row['close']
            tokenAmount = (wallet * leverage) / row['close']
            shortLiquidationPrice = shortIniPrice + (wallet/tokenAmount)
            # -- You can uncomment the line below if you want to see logs --
            # print('Open SHORT', closePrice, '$ the', index, '| Liquidation price :', shortLiquidationPrice)

            # -- Add the trade to DT to analyse it later --
            myrow = {'date': index, 'position': "Open Short", 'reason': 'Open Short Market', 'price': closePrice,
                     'frais': takerFee * wallet * leverage, 'wallet': wallet, 'drawBack': (wallet-lastAth)/lastAth}
            #dt = dt.append(myrow, ignore_index=True)
            dt = pd.concat([dt,pd.DataFrame(myrow, index=[0])], ignore_index=True)


# -- BackTest Analyses --
dt = dt.set_index(dt['date'])
dt.index = pd.to_datetime(dt.index)
dt['resultat%'] = dt['wallet'].pct_change()*100

dt['tradeIs'] = ''
dt.loc[dt['resultat%'] > 0, 'tradeIs'] = 'Good'
dt.loc[dt['resultat%'] < 0, 'tradeIs'] = 'Bad'

iniClose = dfTest.iloc[0]['close']
lastClose = dfTest.iloc[len(dfTest)-1]['close']
holdPercentage = ((lastClose - iniClose)/iniClose)
holdWallet = holdPercentage * leverage * initalWallet
algoPercentage = ((wallet - initalWallet)/initalWallet)
vsHoldPercentage = ((wallet - holdWallet)/holdWallet) * 100

try:
    tradesPerformance = round(dt.loc[(dt['tradeIs'] == 'Good') | (dt['tradeIs'] == 'Bad'), 'resultat%'].sum()
            / dt.loc[(dt['tradeIs'] == 'Good') | (dt['tradeIs'] == 'Bad'), 'resultat%'].count(), 2)
except:
    tradesPerformance = 0
    print("/!\ There is no Good or Bad Trades in your BackTest, maybe a problem...")

try:
    TotalGoodTrades = dt.groupby('tradeIs')['date'].nunique()['Good']
    AveragePercentagePositivTrades = round(dt.loc[dt['tradeIs'] == 'Good', 'resultat%'].sum()
                                           / dt.loc[dt['tradeIs'] == 'Good', 'resultat%'].count(), 2)
    idbest = dt.loc[dt['tradeIs'] == 'Good', 'resultat%'].idxmax()
    bestTrade = str(
        round(dt.loc[dt['tradeIs'] == 'Good', 'resultat%'].max(), 2))
except:
    TotalGoodTrades = 0
    AveragePercentagePositivTrades = 0
    idbest = ''
    bestTrade = 0
    print("/!\ There is no Good Trades in your BackTest, maybe a problem...")

try:
    TotalBadTrades = dt.groupby('tradeIs')['date'].nunique()['Bad']
    AveragePercentageNegativTrades = round(dt.loc[dt['tradeIs'] == 'Bad', 'resultat%'].sum()
                                           / dt.loc[dt['tradeIs'] == 'Bad', 'resultat%'].count(), 2)
    idworst = dt.loc[dt['tradeIs'] == 'Bad', 'resultat%'].idxmin()
    worstTrade = round(dt.loc[dt['tradeIs'] == 'Bad', 'resultat%'].min(), 2)
except:
    TotalBadTrades = 0
    AveragePercentageNegativTrades = 0
    idworst = ''
    worstTrade = 0
    print("/!\ There is no Bad Trades in your BackTest, maybe a problem...")

totalTrades = TotalBadTrades + TotalGoodTrades

try:
    TotalLongTrades = dt.groupby('position')['date'].nunique()['LONG']
    AverageLongTrades = round(dt.loc[dt['position'] == 'LONG', 'resultat%'].sum()
                              / dt.loc[dt['position'] == 'LONG', 'resultat%'].count(), 2)
    idBestLong = dt.loc[dt['position'] == 'LONG', 'resultat%'].idxmax()
    bestLongTrade = str(
        round(dt.loc[dt['position'] == 'LONG', 'resultat%'].max(), 2))
    idWorstLong = dt.loc[dt['position'] == 'LONG', 'resultat%'].idxmin()
    worstLongTrade = str(
        round(dt.loc[dt['position'] == 'LONG', 'resultat%'].min(), 2))
except:
    AverageLongTrades = 0
    TotalLongTrades = 0
    bestLongTrade = ''
    idBestLong = ''
    idWorstLong = ''
    worstLongTrade = ''
    print("/!\ There is no LONG Trades in your BackTest, maybe a problem...")

try:
    TotalShortTrades = dt.groupby('position')['date'].nunique()['SHORT']
    AverageShortTrades = round(dt.loc[dt['position'] == 'SHORT', 'resultat%'].sum()
                               / dt.loc[dt['position'] == 'SHORT', 'resultat%'].count(), 2)
    idBestShort = dt.loc[dt['position'] == 'SHORT', 'resultat%'].idxmax()
    bestShortTrade = str(
        round(dt.loc[dt['position'] == 'SHORT', 'resultat%'].max(), 2))
    idWorstShort = dt.loc[dt['position'] == 'SHORT', 'resultat%'].idxmin()
    worstShortTrade = str(
        round(dt.loc[dt['position'] == 'SHORT', 'resultat%'].min(), 2))
except:
    AverageShortTrades = 0
    TotalShortTrades = 0
    bestShortTrade = ''
    idBestShort = ''
    idWorstShort = ''
    worstShortTrade = ''
    print("/!\ There is no SHORT Trades in your BackTest, maybe a problem...")

try:
    totalGoodLongTrade = dt.groupby(['position', 'tradeIs']).size()['LONG']['Good']
except:
    totalGoodLongTrade = 0
    print("/!\ There is no good LONG Trades in your BackTest, maybe a problem...")

try:
    totalBadLongTrade = dt.groupby(['position', 'tradeIs']).size()['LONG']['Bad']
except:
    totalBadLongTrade = 0
    print("/!\ There is no bad LONG Trades in your BackTest, maybe a problem...")

try:
    totalGoodShortTrade = dt.groupby(['position', 'tradeIs']).size()['SHORT']['Good']
except:
    totalGoodShortTrade = 0
    print("/!\ There is no good SHORT Trades in your BackTest, maybe a problem...")

try:
    totalBadShortTrade = dt.groupby(['position', 'tradeIs']).size()['SHORT']['Bad']
except:
    totalBadShortTrade = 0
    print("/!\ There is no bad SHORT Trades in your BackTest, maybe a problem...")

TotalTrades = TotalGoodTrades + TotalBadTrades
winRateRatio = (TotalGoodTrades/TotalTrades) * 100

reasons = dt['reason'].unique()

print("BackTest finished, final wallet :",wallet,"$")
#dt


<h1> Print and show results </h1>

In [ ]:
print("Pair Symbol :","BTC/USDT",)
print("Period : [" + str(dfTest.index[0]) + "] -> [" +
      str(dfTest.index[len(dfTest)-1]) + "]")
print("Starting balance :", initalWallet, "$")

print("\n----- General Informations -----")
print("Final balance :", round(wallet, 2), "$")
print("Hold balance :", round(holdWallet, 2), "$")
print("Performance vs US Dollar :", round(algoPercentage*100, 2), "%")
print("Buy and Hold Performence :", round(holdPercentage*100, 2),
      "% | with Leverage :", round(holdPercentage*100, 2)*leverage, "%")
print("Performance vs Buy and Hold :", round(vsHoldPercentage, 2), "%")
print("Best trade : +"+bestTrade, "%, the ", idbest)
print("Worst trade :", worstTrade, "%, the ", idworst)
print("Worst drawBack :", str(100*round(dt['drawBack'].min(), 2)), "%")
print("Total fees : ", round(dt['frais'].sum(), 2), "$")

print("\n----- Trades Informations -----")
print("Total trades on period :",totalTrades)
print("Number of positive trades :", TotalGoodTrades)
print("Number of negative trades : ", TotalBadTrades)
print("Trades win rate ratio :", round(winRateRatio, 2), '%')
print("Average trades performance :",tradesPerformance,"%")
print("Average positive trades :", AveragePercentagePositivTrades, "%")
print("Average negative trades :", AveragePercentageNegativTrades, "%")

print("\n----- LONG Trades Informations -----")
print("Number of LONG trades :",TotalLongTrades)
print("Average LONG trades performance :",AverageLongTrades, "%")
print("Best  LONG trade +"+bestLongTrade, "%, the ", idBestLong)
print("Worst LONG trade", worstLongTrade, "%, the ", idWorstLong)
print("Number of positive LONG trades :",totalGoodLongTrade)
print("Number of negative LONG trades :",totalBadLongTrade)
print("LONG trade win rate ratio :", round(totalGoodLongTrade/TotalLongTrades*100, 2), '%')

print("\n----- SHORT Trades Informations -----")
print("Number of SHORT trades :",TotalShortTrades)
print("Average SHORT trades performance :",AverageShortTrades, "%")
print("Best  SHORT trade +"+bestShortTrade, "%, the ", idBestShort)
print("Worst SHORT trade", worstShortTrade, "%, the ", idWorstShort)
print("Number of positive SHORT trades :",totalGoodShortTrade)
print("Number of negative SHORT trades :",totalBadShortTrade)
print("SHORT trade win rate ratio :", round(totalGoodShortTrade/TotalShortTrades*100, 2), '%')

print("\n----- Trades Reasons -----")
reasons = dt['reason'].unique()
for r in reasons:
    print(r+" number :", dt.groupby('reason')['date'].nunique()[r])
    
dt[['wallet', 'price']].plot(subplots=True, figsize=(20, 10))
print("\n----- Plot -----")